<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/tasca1/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>